In [1]:
# write your code
# ref: https://www.dataquest.io/blog/web-scraping-beautifulsoup/ 
# ref: https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/ 
    
import requests
from requests import get
from bs4 import BeautifulSoup
import urllib.request # to download images
import re
import os # to make a directory
import csv



In [2]:
page = requests.get("https://www.petfinder.com/dog/laila-54754243/bc/prince-george/bc-spca-north-cariboo-branch-prince-george-bc20/")
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
dog_name = soup.find(attrs={"data-test" : "Pet_Name"})
dog_name = re.sub('\s+', '', dog_name.contents[0]).lower() # remove \n, tabs and spaces from string

images = []
for image in soup.find_all('img'):
    images.append(image['src'])

substring = '/'
img_counter = 1
unique_images = []

for i in range(len(images)):
    substring = substring + str(img_counter) + "/" 
    if(substring in images[i]):
        unique_images.append(images[i])
    substring = '/'
    img_counter = img_counter + 1



In [4]:
unique_dog_dir = ''
unique_dog_dir = unique_dog_dir + dog_name

dog_breed = soup.find(attrs={"data-test" : "Pet_Breeds"}).text.strip()

unique_dog_dir = unique_dog_dir + '-' + dog_breed.lower().replace(" ", "-") # breed to dogname

pet_location = soup.find(attrs={"data-test" : "Pet_Location"}).text.strip()

dog_location =  pet_location.split(",")
dog_city = dog_location[0]
dog_state = dog_location[1]

pet_location_dir = re.sub('\s+', '', pet_location).lower() # remove \n, tabs and spaces from string

unique_dog_dir = unique_dog_dir + '-' + pet_location_dir.replace(",", "-") # location to dogname

dog_location =  pet_location.split(",")
dog_city = dog_location[0]
dog_state = dog_location[1]

dog_age = soup.find(attrs={"data-test" : "Pet_Age"}).text.strip()
dog_sex = soup.find(attrs={"data-test" : "Pet_Sex"}).text.strip()
dog_size = soup.find(attrs={"data-test" : "Pet_Full_Grown_Size"}).text.strip()


print("dog_name: ", dog_name)
print("dog_breed: ", dog_breed)

print("dog_city: ", dog_city)
print("dog_state: ", dog_state)
print("unique_dog_dir: ", unique_dog_dir)

print("dog_age: ", dog_age)
print("dog_sex: ", dog_sex)
print("dog_size: ", dog_size)

dog_name:  laila
dog_breed:  Pit Bull Terrier Mix
dog_city:  Prince George
dog_state:   BC
unique_dog_dir:  laila-pit-bull-terrier-mix-princegeorge-bc
dog_age:  Young
dog_sex:  Female
dog_size:  Medium


In [5]:
unique_directory_counter = 1

In [6]:

try:
    path = os.path.join("images/", unique_dog_dir)
    os.makedirs(path)
except FileExistsError:
    # directory already exists - append unique num at the end of dir to make it unique
    path = os.path.join("images/", unique_dog_dir + '-' + str(unique_directory_counter))
    os.makedirs(path)
    unique_directory_counter = unique_directory_counter + 1
    pass

In [7]:
# download dog's images in that unique directory
unique_images_counter = 1
for image_url in unique_images:
    # urllib.request.urlretrieve(image_url, "try_images/try.jpg")
    dog_image_name = dog_name + '-' + str(unique_images_counter) + ".jpg"
    dog_image_path = path + '/' + dog_image_name
    
    urllib.request.urlretrieve(image_url, dog_image_path)
    
    unique_images_counter = unique_images_counter + 1
    print(image_url)
    print(dog_image_path)

https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/54754243/1/?bust=1645813108&width=720
images/laila-pit-bull-terrier-mix-princegeorge-bc/laila-1.jpg
https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/54754243/2/?bust=1645813108&width=720
images/laila-pit-bull-terrier-mix-princegeorge-bc/laila-2.jpg


In [8]:
#ref: https://www.pythontutorial.net/python-basics/python-write-csv-file/

header = ['dog_name', 'dog_breed', 'dog_city', 'dog_state', 'unique_dog_dir', 'dog_age', 'dog_sex', 'dog_size', 'dog_images']
data = [dog_name, dog_breed, dog_city, dog_state, unique_dog_dir, dog_age, dog_sex, dog_size, len(unique_images)]

csv_file_name = dog_name + '.csv'

with open(path + '/' + csv_file_name, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerow(data)